In [4]:
import httpx
import asyncio
from typing import Dict, Any, TypedDict, Callable

# Why use callables in Functional Programming?

The more I learn about Functional Programming (**FP**) the more I like it. I find it much code written in a more FP style much easier to debug and undertand. This post discusses something I first learned about in [Grokking Simplicity](https://learning.oreilly.com/library/view/grokking-simplicity/9781617296208/): passing callables to a function, rather than calling the sub-function directly. I didn't understand why passing the callable is a much better approach, until I came across it myself.

TLDR: it's more modular, and easier to test.

To better explain, in my case I have a function to make a `GET` request and return the response and another function for error handling. Initially, here are those two functions.

In [2]:
async def get_url(
    url: str, client: httpx.AsyncClient, limiter: asyncio.Semaphore = None
) -> httpx.Response:
    """Send and async request to the given URL using the client

    Allows for the optimal use of a Semaphore to restrict simultaneous Async calls.

    Parameters
    ---------
    url : str
        the url to get
    client : httpx.AsyncClient
        the httpx client to use for the call
    limiter : asyncio.Semaphore
        the Semaphore instance to use. Optional
    
    Returns
    -------
    httpx.Response
        the reponse object from the `url`
    """
    assert isinstance(client, httpx.AsyncClient)
    "`client` must be an httpx.AsyncClient"
    if not limiter:
        response = await client.get(url)
    else:
        async with limiter:
            response = await client.get(url)
    response.raise_for_status()
    return response

class GetterArgs(TypedDict):
    url: str
    client: httpx.AsyncClient
    limiter: asyncio.Semaphore | None

async def get_response(
    getter_args: GetterArgs,
    retries: int = 10,
) -> httpx.Response:
    '''Handles errors that are returned from `get_url`
    
    Parameters
    ----------
    getter_args : GetterArgs
        the arguments to be passed to `get_url`
    retries : int
        the number of times to retry a call, for calls that should be retried

    Returns
    -------
    httpx.Response
        the response from the url
    '''

    try:
        response: httpx.Response = await get_url(**getter_args)
    except httpx.HTTPStatusError as e:
        code = response.status_code
        if code == 429 or (code >= 100 and code < 200):
            if "try_count" not in locals():
                try_count = 1
            else:
                try_count += 1
            if try_count >= retries and retries >= 0:
                print("Max retries exceeded for 429 error")
                raise e
            response = await get_url(**getter_args)
        else:
            raise e
        # TODO: account for other error codes; e.g. redirections
    return response


You can see that I've defined `get_url` and `get_reponse`. I call `get_url` from `get_response` and this approach works just fine.

In [3]:
async with httpx.AsyncClient() as client:
    getter_args = GetterArgs(
        url='http://httpbin.org/get',
        client=client
    )
    response = await get_response(getter_args=getter_args)
assert response.status_code == 200
print('Success')

Success


Exactly as intended. The tricky part is testing `get_response`. Among other things, I wanted to test whether a 429 error would cause `get_response` to retry the correct number of times. As it stands, I could mock a new function to replace `get_url`, which would likely achieve my goal. But it would be much cleaner to abstract the call to `get_url` by passing the function to `get_response` as an argument.

In [5]:
async def get_response(
    getter: Callable,
    getter_args: GetterArgs,
    retries: int = 10,
) -> httpx.Response:
    '''Handles errors that are returned from `get_url`
    
    Parameters
    ----------
    getter : Callable
        the function to use to get the data
    getter_args : GetterArgs
        the arguments to be passed to `get_url`
    retries : int
        the number of times to retry a call, for calls that should be retried

    Returns
    -------
    httpx.Response
        the response from the url
    '''
    try:
        response: httpx.Response = await getter(**getter_args)
    except httpx.HTTPStatusError as e:
        code = response.status_code
        if code == 429 or (code >= 100 and code < 200):
            if "try_count" not in locals():
                try_count = 1
            else:
                try_count += 1
            if try_count >= retries and retries >= 0:
                print("Max retries exceeded for 429 error")
                raise e
            response = await getter(**getter_args)
        else:
            raise e
        # TODO: account for other error codes; e.g. redirections
    return response

This code follows FP principles much better because it is composable and is independent from the getter. In addition to being easier to test, `get_response` won't be affected by changes to the getter function name or arguments. The only requirement is that the getter implements the correct interface.

In [39]:
# Define a function to replace `get_url`
class MockGetter:

    def __init__(self) -> None:
        self.count_retries = 0
        
    def getter(self): 
        self.count_retries += 1
        resp = httpx.get('http://httpbin.org/status/429')
        resp.raise_for_status()

In [40]:
def test_429_error():
    mg = MockGetter()
    retries = 10
    get_response(getter=mg.getter, getter_args={}, retries=retries)
    assert mg.count_retries == retries, f"There should have been {retries} retries, but there were {mg.count_retries}"

test_429_error()

/tmp/ipykernel_2802/1025198436.py:4: RuntimeWarning: coroutine 'get_response' was never awaited
  get_response(getter=mg.getter, getter_args={}, retries=retries)


AssertionError: There should have been 10 retries, but there were 0

In [41]:
mg = MockGetter()

In [42]:
mg.getter()

HTTPStatusError: Client error '429 TOO MANY REQUESTS' for url 'http://httpbin.org/status/429'
For more information check: https://httpstatuses.com/429